Сегодня мы поговорим о морфологических анализаторах для русского языка. Важно отметить, что (в основном) для каждого естественного языка нужен свой морфоанализатор.

* шаг 1: установим пайморфи и майстем

In [1]:
!pip install pymystem3==0.1.10
!pip install pymorphy2[fast]

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
# импорты
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

# сохраняем класс в переменную
mystem = Mystem() 
morph = MorphAnalyzer() 

In [3]:
# сэмпл-текст, на котором все будем пробовать
text = """Система состоит из камеры и программного обеспечения, которое анализирует фотографию.
 Суть технологии — сопоставление лиц, попавших в объектив, с изображениями из базы данных"""

## нормализация

давайте приведем текст к нижнему регистру

In [4]:
text = text.lower()
text

'система состоит из камеры и программного обеспечения, которое анализирует фотографию.\n суть технологии — сопоставление лиц, попавших в объектив, с изображениями из базы данных'

## токенизация

иногда текст нужно предварительно разбить на токены, отдельные элементы. Токенизаторов много, каждый делает это, руководствуясь своими правилами

* токенизация нужна, если морофоанализатор не умеет токенизировать сам (Mystem умеет, pymorphy не умеет)

In [5]:
# попробуем токенизацию от модуля gensim
from gensim.utils import tokenize

list(tokenize(text))

['система',
 'состоит',
 'из',
 'камеры',
 'и',
 'программного',
 'обеспечения',
 'которое',
 'анализирует',
 'фотографию',
 'суть',
 'технологии',
 'сопоставление',
 'лиц',
 'попавших',
 'в',
 'объектив',
 'с',
 'изображениями',
 'из',
 'базы',
 'данных']

In [6]:
#  еще можно токенизировать функциями из модуля nltk

from nltk.tokenize import word_tokenize, wordpunct_tokenize # попробуем две функции

In [7]:
wordpunct_tokenize("O'Brian, Ростов-на-Дону, обычноеслово")

['O', "'", 'Brian', ',', 'Ростов', '-', 'на', '-', 'Дону', ',', 'обычноеслово']

In [17]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context


import nltk
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

True

In [18]:

word_tokenize("O'Brian, Ростов-на-Дону, обычноеслово")

["O'Brian", ',', 'Ростов-на-Дону', ',', 'обычноеслово']

## лемматизация и морфоанализ

### Mystem


Майстем работает немного лучше и сам токенизирует,
поэтому можно ему подавать сырой текст.

In [19]:
# сначала лемматизируем слова методом .lemmatize()
print(text)
print(mystem.lemmatize(text))


система состоит из камеры и программного обеспечения, которое анализирует фотографию.
 суть технологии — сопоставление лиц, попавших в объектив, с изображениями из базы данных
['система', ' ', 'состоять', ' ', 'из', ' ', 'камера', ' ', 'и', ' ', 'программный', ' ', 'обеспечение', ', ', 'который', ' ', 'анализировать', ' ', 'фотография', '.', '\n', ' ', 'суть', ' ', 'технология', ' — ', 'сопоставление', ' ', 'лицо', ', ', 'попадать', ' ', 'в', ' ', 'объектив', ', ', 'с', ' ', 'изображение', ' ', 'из', ' ', 'база', ' ', 'данные', '\n']


In [22]:
# метод .analyze() даст грамматическую информацию о словах
words_analyzed = mystem.analyze(text)

этот метод возвращает список словарей


каждый словарь имеет либо одно поле 'text' (когда попался пробел или пунктуация) или text и analysis

* в analysis снова список словарей с вариантами разбора (первый самый вероятный)
* поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
* analysis - может быть пустым списком

In [30]:
print("объект типа", type(words_analyzed),"\n",
      "длина: ", len(words_analyzed),"\n",
      words_analyzed) # посмотрим на анализ и теги

объект типа <class 'list'> 
 длина:  46 
 [{'analysis': [{'lex': 'система', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}], 'text': 'система'}, {'text': ' '}, {'analysis': [{'lex': 'состоять', 'wt': 1, 'gr': 'V,несов,нп=непрош,ед,изъяв,3-л'}], 'text': 'состоит'}, {'text': ' '}, {'analysis': [{'lex': 'из', 'wt': 1, 'gr': 'PR='}], 'text': 'из'}, {'text': ' '}, {'analysis': [{'lex': 'камера', 'wt': 1, 'gr': 'S,жен,неод=(вин,мн|род,ед|им,мн)'}], 'text': 'камеры'}, {'text': ' '}, {'analysis': [{'lex': 'и', 'wt': 0.9999770522, 'gr': 'CONJ='}], 'text': 'и'}, {'text': ' '}, {'analysis': [{'lex': 'программный', 'wt': 1, 'gr': 'A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)'}], 'text': 'программного'}, {'text': ' '}, {'analysis': [{'lex': 'обеспечение', 'wt': 1, 'gr': 'S,сред,неод=(вин,мн|род,ед|им,мн)'}], 'text': 'обеспечения'}, {'text': ', '}, {'analysis': [{'lex': 'который', 'wt': 1, 'gr': 'APRO=(вин,ед,сред|им,ед,сред)'}], 'text': 'которое'}, {'text': ' '}, {'analysis': [{'lex': 'анализировать

In [31]:
# сделаем все красиво с индексами и доступом по ключам

print('Слово - ', words_analized[0]['text'])
print('Разбор слова - ', words_analized[0]['analysis'][0])
print('Лемма слова - ', words_analized[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', words_analized[0]['analysis'][0]['gr'])

Слово -  система
Разбор слова -  {'lex': 'система', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}
Лемма слова -  система
Грамматическая информация слова -  S,жен,неод=им,ед


In [32]:
#леммы можно достать в одну строчку
[parse['analysis'][0]['lex'] for parse in words_analized if parse.get('analysis')][:10]

['система',
 'состоять',
 'из',
 'камера',
 'и',
 'программный',
 'обеспечение',
 'который',
 'анализировать',
 'фотография']

In [33]:
# то же самое, что в предыдущей ячейке, но циклом
res = []
for parse in words_analized:
    if parse.get('analysis'):
        res.append(parse['analysis'][0]['lex'])

res[:10]

['система',
 'состоять',
 'из',
 'камера',
 'и',
 'программный',
 'обеспечение',
 'который',
 'анализировать',
 'фотография']

#### дополнительные возможности Mystem

Mystem умеет разбивать текст на предложения, но через питоновский интерфейс это сделать не получится. Нужно скачать mystem отсюда - https://yandex.ru/dev/mystem/ и использовать в командной строке

Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

### Pymorphy

Pymorphy - открытый и развивается (можно поучаствовать на гитхабе)

Ссылка на репозиторий: https://github.com/kmike/pymorphy2

Попробуйте сразу установить быструю версию (команда для терминала ```pip3 install pymorphy2[fast]```)

* [документация pymorphy](https://pythonhosted.org/pymorphy/)

У него нет втстроенной токенизации и он расценивает всё как слово. Когда есть несколько вариантов, он выдает их с вероятностостями, которые расчитатны на корпусе со снятой неоднозначностью. Это лучше стемминга, но хуже майстема.

In [42]:
# основная функция - pymorphy.parse
words_analyzed = [morph.parse(token) for token in word_tokenize(text)]

In [43]:
words_analyzed

[[Parse(word='система', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='система', score=1.0, methods_stack=((DictionaryAnalyzer(), 'система', 55, 0),))],
 [Parse(word='состоит', tag=OpencorporaTag('VERB,impf,intr sing,3per,pres,indc'), normal_form='состоять', score=1.0, methods_stack=((DictionaryAnalyzer(), 'состоит', 2269, 5),))],
 [Parse(word='из', tag=OpencorporaTag('PREP'), normal_form='из', score=0.999673, methods_stack=((DictionaryAnalyzer(), 'из', 393, 0),)),
  Parse(word='из', tag=OpencorporaTag('NOUN,anim,femn,Name plur,gent'), normal_form='иза', score=0.000163, methods_stack=((DictionaryAnalyzer(), 'из', 69, 8),)),
  Parse(word='из', tag=OpencorporaTag('NOUN,anim,femn,Name plur,accs'), normal_form='иза', score=0.000163, methods_stack=((DictionaryAnalyzer(), 'из', 69, 10),))],
 [Parse(word='камеры', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='камера', score=0.484848, methods_stack=((DictionaryAnalyzer(), 'камеры', 55, 1),)),
  Parse(word='камеры',

In [45]:
morph.parse("печь") # пример с морфологической неоднозначностью

[Parse(word='печь', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='печь', score=0.571428, methods_stack=((DictionaryAnalyzer(), 'печь', 2223, 0),)),
 Parse(word='печь', tag=OpencorporaTag('INFN,impf,tran'), normal_form='печь', score=0.285714, methods_stack=((DictionaryAnalyzer(), 'печь', 2456, 0),)),
 Parse(word='печь', tag=OpencorporaTag('NOUN,inan,femn sing,accs'), normal_form='печь', score=0.142857, methods_stack=((DictionaryAnalyzer(), 'печь', 2223, 3),))]

Она похожа на analyze в майстеме только возвращает список объектов Parse
* Первый в списке - самый вероятный разбор (у каждого есть score)
* Информация достается через атрибут (Parse.word)
* Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
части речи или другие категории

In [47]:
# сделаем красиво и тут
print('Первое слово - ', words_analyzed[0][0].word)
print('Разбор первого слова - ', words_analyzed[0][0])
print('Лемма первого слова - ', words_analyzed[0][0].normal_form)
print('Грамматическая информация первого слова - ', words_analyzed[0][0].tag)
print('Часть речи первого слова - ', words_analyzed[0][0].tag.POS)
print('Род первого слова - ', words_analyzed[0][0].tag.gender)
print('Число первого слова - ', words_analyzed[0][0].tag.number)
print('Падеж первого слова - ', words_analyzed[0][0].tag.case)

Первое слово -  система
Разбор первого слова -  Parse(word='система', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='система', score=1.0, methods_stack=((DictionaryAnalyzer(), 'система', 55, 0),))
Лемма первого слова -  система
Грамматическая информация первого слова -  NOUN,inan,femn sing,nomn
Часть речи первого слова -  NOUN
Род первого слова -  femn
Число первого слова -  sing
Падеж первого слова -  nomn


### что можно дальше

Pymorphy и Mystem - не единственные морфоанализаторы для русского языка. Можно, например, посмотреть на [RNNmorph](https://github.com/IlyaGusev/rnnmorph) и [deeppavlov](http://docs.deeppavlov.ai/en/master/features/models/morphotagger.html).

А еще есть исследование, где сравнивали морфоанализаторы для русского  ([краткая версия](http://web-corpora.net/wsgi/mystemplus.wsgi/mystemplus/compare_table/), [статья](http://www.dialog-21.ru/media/3473/dereza.pdf))

 